In [3]:
import pandas as pd
import numpy as np

In [4]:
test = pd.read_csv('/home/soniya51/Toxic Comments/test.csv')

In [5]:
train = pd.read_csv('/home/soniya51/Toxic Comments/train.csv')

In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
train.isnull().any(),test.isnull().any()

(id               False
 comment_text     False
 toxic            False
 severe_toxic     False
 obscene          False
 threat           False
 insult           False
 identity_hate    False
 dtype: bool, id              False
 comment_text    False
 dtype: bool)

In [8]:
import keras
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [9]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values

In [10]:
sentences_train = train["comment_text"]
sentences_test = test["comment_text"]
type(sentences_train[1])

str

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(sentences_train))
tokenized_train = tokenizer.texts_to_sequences(sentences_train)
tokenized_test = tokenizer.texts_to_sequences(sentences_test)


In [12]:
maxlen = 200
X_t = pad_sequences(tokenized_train, maxlen=maxlen)
X_te = pad_sequences(tokenized_test, maxlen=maxlen)

In [13]:
#200 is the length of each padded array
'''
def divide_chunks(l, n): 
    for i in range(0, len(l), n): 
        yield l[i:i + n] 
def pad(int_comments):
    for single_comment in int_comments:
        if(len(single_comment) > 200):
            single_comment = list(divide_chunks(single_comment, 200))
            n = len(single_comment)
            single_comment[n-1] += [0] * (200 - len(single_comment[n-1]))
        else:
            single_comment += [0] * (200 - len(single_comment))
    return int_comments
'''

'\ndef divide_chunks(l, n): \n    for i in range(0, len(l), n): \n        yield l[i:i + n] \ndef pad(int_comments):\n    for single_comment in int_comments:\n        if(len(single_comment) > 200):\n            single_comment = list(divide_chunks(single_comment, 200))\n            n = len(single_comment)\n            single_comment[n-1] += [0] * (200 - len(single_comment[n-1]))\n        else:\n            single_comment += [0] * (200 - len(single_comment))\n    return int_comments\n'

In [14]:
inp = Input(shape=(maxlen, ))

In [15]:
!ls

 anaconda3
 a.out
 CDLab
 codeforces
 codeforces.cpp
 cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
 Desktop
 Documents
 Downloads
 glove.6B.100d.txt
 glove.6B.200d.txt
 glove.6B.300d.txt
 glove.6B.50d.txt
 Music
'Physical registration'
 Pictures
 Public
 PycharmProjects
 Repos
 snap
 Templates
'Toxic Comments'
 ToxicComments.ipynb
 Videos


In [16]:
MAX_NB_WORDS = 100000    # max no. of words for tokenizer
MAX_SEQUENCE_LENGTH = 200 # max length of each entry (sentence), including padding
VALIDATION_SPLIT = 0.2   # data for validation (not used in training)
EMBEDDING_DIM = 100      # embedding dimensions for word vectors (word2vec/GloVe)
GLOVE_DIR = "glove.6B."+str(EMBEDDING_DIM)+"d.txt"

In [17]:
word_index = tokenizer.word_index
print((word_index['name']))

111


In [18]:
embeddings_index = {}
f = open(GLOVE_DIR)
print('Loading GloVe from:', GLOVE_DIR,'...', end='')
for line in f:
    values = line.split()
    word = values[0]
    embeddings_index[word] = np.asarray(values[1:], dtype='float32')
f.close()
print("Done.\n Proceeding with Embedding Matrix...", end="")

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print(" Completed!")

Loading GloVe from: glove.6B.100d.txt ...Done.
 Proceeding with Embedding Matrix... Completed!


In [19]:
embedding_layer = Embedding(len(word_index) + 1,
                           EMBEDDING_DIM,
                           weights = [embedding_matrix],
                           input_length = MAX_SEQUENCE_LENGTH,
                           trainable=False,
                           name = 'embeddings')
embedded_sequences = embedding_layer(inp)

In [20]:
x = LSTM(60, return_sequences=True,name='lstm_layer')(embedded_sequences)

In [21]:
x = GlobalMaxPool1D()(x)

In [22]:
x = Dropout(0.1)(x)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [23]:
x = Dense(50, activation="relu")(x)

In [24]:
x = Dropout(0.1)(x)

In [25]:
preds = Dense(6, activation="sigmoid")(x)

In [26]:
model = Model(inputs=inp, outputs=preds)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [27]:
batch_size = 32
epochs = 2
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)



Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 684s 5ms/step - loss: 0.0666 - acc: 0.9773 - val_loss: 0.0533 - val_acc: 0.9804
Epoch 2/2
143613/143613 [==============================] - 677s 5ms/step - loss: 0.0508 - acc: 0.9814 - val_loss: 0.0497 - val_acc: 0.9816


In [28]:
print(preds)

Tensor("dense_2/Sigmoid:0", shape=(?, 6), dtype=float32)


In [35]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission = pd.read_csv("../soniya51/Downloads/sample_submission.csv")
sample_submission[list_classes] = y_test
sample_submission.to_csv('submission.csv', index=False)

153164/153164 [==============================] - 15s 100us/step


In [41]:
from csv import reader
with open("submission.csv","r") as f:
    reader = csv.reader(f,delimiter = ",")
    data = list(reader)
    row_count = len(data)
    row_count

NameError: name 'csv' is not defined

In [42]:
import pandas as pd

results = pd.read_csv('submission.csv')

print(len(results))

153164


In [43]:
results.head()


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.985254,0.234086,0.920320,0.060797,0.825374,0.106350
1,0000247867823ef7,0.001496,0.000002,0.000231,0.000007,0.000151,0.000019
2,00013b17ad220c46,0.002196,0.000009,0.000394,0.000029,0.000285,0.000061
3,00017563c3f7919a,0.001842,0.000003,0.000319,0.000008,0.000201,0.000016
4,00017695ad8997eb,0.003612,0.000015,0.000656,0.000031,0.000465,0.000064
